In [64]:
from pathlib import Path
import pandas as pd

from top2vec import Top2Vec

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.optimizers import SGD

In [2]:
path_corpora = Path('data/corpora')

path_corpus_tut = path_corpora / Path('tutby_126784.csv')

path_model_top2vec = Path('data/model/top2vec_126784.joblibfile')

In [3]:
data = pd.read_csv(path_corpus_tut)
data['tags'] = data['tags'].apply(eval)

print(data.shape)
display(data.head(3))

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,[футбол]
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"[архитектура, живопись, ЖКХ]"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,[министерства]


In [4]:
data['tags'].explode().value_counts()

криминал                  9061
правовая информация       8307
дети                      7748
министерства              7745
милиция                   6372
                          ... 
экономия воды                1
перекрытие дома              1
Крушение A320 EgyptAir       1
автоблоги                    1
шумоизоляция                 1
Name: tags, Length: 1282, dtype: int64

In [5]:
model = Top2Vec.load(path_model_top2vec)

x = model.model.docvecs.vectors_docs

print(x.shape)

(126784, 300)


In [6]:
label_binarizer = MultiLabelBinarizer()

y = label_binarizer.fit_transform(data['tags'])

print(len(label_binarizer.classes_))
print(y.shape)

1282
(126784, 1282)


In [72]:
%%time

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=x.shape[1]))
model.add(Dense(y.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.fit(x_train, y_train, epochs=6, batch_size=32, validation_split=0.2)

pred = model.predict(x_test)

pred = pred > 0.5

Train on 67956 samples, validate on 16989 samples
Epoch 1/6
67956/67956 [==============================] - 16s 231us/step - loss: 0.0157 - categorical_accuracy: 0.1831 - val_loss: 0.0075 - val_categorical_accuracy: 0.2726
Epoch 2/6
67956/67956 [==============================] - 15s 224us/step - loss: 0.0068 - categorical_accuracy: 0.2991 - val_loss: 0.0064 - val_categorical_accuracy: 0.3243
Epoch 3/6
67956/67956 [==============================] - 15s 218us/step - loss: 0.0061 - categorical_accuracy: 0.3268 - val_loss: 0.0061 - val_categorical_accuracy: 0.3274
Epoch 4/6
67956/67956 [==============================] - 15s 217us/step - loss: 0.0057 - categorical_accuracy: 0.3405 - val_loss: 0.0059 - val_categorical_accuracy: 0.3263
Epoch 5/6
67956/67956 [==============================] - 15s 218us/step - loss: 0.0055 - categorical_accuracy: 0.3475 - val_loss: 0.0059 - val_categorical_accuracy: 0.3438
Epoch 6/6
67956/67956 [==============================] - 15s 221us/step - loss: 0.0054 - c

In [85]:
y_test_label = label_binarizer.inverse_transform(y_test)
pred_label = label_binarizer.inverse_transform(pred)

for a, b in zip(pred_label[:10], y_test_label[:10]):
    print(a, ' --- ', b)

()  ---  ('Forbes',)
('В мире', 'Дело отравленного шпиона')  ---  ('В мире', 'Дело отравленного шпиона')
('вело',)  ---  ('ЗОЖ-индустрия', 'здоровый образ жизни')
('право', 'правовая информация')  ---  ('Видео TUT.BY', 'Евросоюз', 'Простая политика')
('животные',)  ---  ('Жестокое обращение с животными',)
()  ---  ('Банкротства', 'банки', 'суды')
('Зарплаты', 'деньги', 'личные финансы', 'соцзащита', 'статистика')  ---  ('деньги',)
('Транспорт', 'дети', 'строительство', 'туризм')  ---  ('Города-спутники', 'детские сады', 'жилье', 'строительство')
('ДТП', 'пешеходы')  ---  ('ДТП', 'пешеходы')
('Здравоохранение', 'Образование', 'благотворительность', 'дети', 'общество', 'семья')  ---  ('банки', 'благотворительность', 'дети', 'здоровье', 'история Беларуси', 'многоборье', 'происшествия', 'семья', 'социальная неустроенность', 'спорт')


In [82]:
accuracy_score(y_test, pred)

0.14015631348741606